In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg
from matplotlib import colors

In [ ]:
margin = 0.5

vector_params = {
    'color': ['b', 'r'],
    'width': 0.013,
    'angles': 'xy',
    'scale_units': 'xy',
    'scale': 1
}

space_names = [
    "original",
    "transformed"
]

In [ ]:
def get_unit_circle(n=100):
    xi1 = np.linspace(-1.0, 1.0, n)
    xi2 = np.linspace(1.0, -1.0, n)
    yi1 = np.sqrt(1 - xi1**2)
    yi2 = -np.sqrt(1 - xi2**2)

    xi = np.concatenate((xi1, xi2),axis=0)
    yi = np.concatenate((yi1, yi2),axis=0)
    return np.vstack((xi, yi))

In [ ]:
def plot_2d_vectors(vs, f_vs, shapes=None):
    origin = np.array([[0, 0]])
    vss = np.vstack([origin, vs, f_vs, *shapes])
    fig, axs = plt.subplots(1, 2, figsize=(10,15))
    plt.subplots_adjust(wspace=0.4)

    if shapes:
        axs[0].plot(shapes[0][:,0], shapes[0][:,1])
        axs[1].plot(shapes[1][:,0], shapes[1][:,1])

    for v, f_v, i in zip(vs, f_vs, range(len(vs))):
        axs[0].quiver(*origin[0], *v, **vector_params)
        axs[1].quiver(*origin[0], *f_v, **vector_params)
        axs[0].text(*v, "$\mathbf{x_i}$".replace('i', str(i)), fontsize=11)
        axs[1].text(*f_v, "$\mathbf{f(x_i)}$".replace('i', str(i)), fontsize=11)

    for ax, space_name in zip(axs, space_names):
        ax.set_xlabel('x', fontsize=14)
        ax.set_ylabel('y', fontsize=14)
        ax.set_xlim(
            np.min(vss[:,0]) - margin, np.max(vss[:,0]) + margin)
        ax.set_ylim(
            np.min(vss[:,1]) - margin, np.max(vss[:,1]) + margin)
        ax.set_aspect('equal')
        ax.grid(True)
        ax.set_axisbelow(True)
        ax.set_title(space_name)
        ax.axhline(y=0, color='k')
        ax.axvline(x=0, color='k')

## 1. Transformaciones lineales

In [ ]:
A = np.array([[1., 2.],
              [0., 1.],
              [1., 1.],
              [0., 0.]])

In [ ]:
x = np.array([[1, 1],
              [2, 2]])

In [ ]:
A @ x

### Giro 45º

In [ ]:
xs = np.array([[1, 0], [0, 1], [1, 1]/np.sqrt(2)])
A = np.array([[1, -1], [1, 1]])/np.sqrt(2)
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

In [ ]:
eig_values, eig_vectors = linalg.eig(A)
print(np.round(eig_values, 4))
print(np.round(eig_vectors, 4))

In [ ]:
xs = np.array([[1, 0], [0, 1], [0, -1], [-1, 0]])
A = np.array([[1, 2], [3, 1]])
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

(!) La matriz de autovalores que devuelve `eig_vectors` está por columnas

## 2. Diagonalización de matrices

Aquí no nos vamos a parar mucho, pero básicamente... cualquier matriz puede ser vista como una aplicación lineal, entonces la movida va de encontar direcciones que no cambian (si existen, o al menos sin irnos al cuerpo de los números complejos $\mathbb{C}$.

Si existen, pillas esas direcciones, montas una base con ellas, y la matriz de la transformación en esa base (o tras aplicar un giro), puede llegar a quedarte diagonal :).

Por ejemplo, en este caso la cosa más o menos funciona bien:

In [ ]:
A = [[1,0], [1,1]]
eig_values, eig_vectors = linalg.eig(A)
print(np.round(eig_values, 4))
print(np.round(eig_vectors, 4))

En este, que es el del un giro, se lía parda, porque nadda más que se queda fijo el origen:

In [ ]:
A = np.array([[1, -1], [1, 1]])/np.sqrt(2)
print(np.round(eig_values, 4))
print(np.round(eig_vectors, 4))

### Autovalores de A

Son los autovalores de A, direcciones invariantes, pero que no son necesriamente ortogonales, ni corresponden con los ejes mayores de la AS.

In [ ]:
A = np.array([[1, 2], [3, 1]])
eig_values, eig_vectors = linalg.eig(A)
xs = eig_vectors.T
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
print(np.round(eig_values, 4))
print(np.round(eig_vectors, 4))

Aquí vemos que puede salir cualquier cosa:

In [ ]:
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

## 3. $AA^T$ y $A^TA$

### Las matrices AA.T y A.TA son simetricas.

Estas matrices son simétricas, y como toda matriz simétrica, tiene autovalores no negativos, y sus autovectores son ortogonales.

Además, sus autovectores coinciden con las direcciones principales (autovalores de mayor valor), por tanto ambas formas simétricas comparten autovalores.

### Matrices simétricas

Las matrices simétricas son aquellas matrices cuadradas que coincidenc con su traspuesta. Tienen algunas propiedades interesantes:

* Si una matriz es diagonalizable ortogonalmente: ($A=QBQ^T$) $\iff$ es simétrica.
* Si sus valores son reales, sus autovalores son reales y sus autovectores ortogonales entre sí.

A symmetric matrix transforms a vector by stretching or shrinking it along its eigenvectors.

### Matrices ortogonales

* Una matriz $A$ es ortogonal si $AA^T=I$.
* Una matriz simétrica no tiene por qué ser ortogonal, pero sus autovectores ... eso si forman una base ortogonal.

### Cambio de base

* $P_{B \rightarrow B_C}$: Vectores de la nueva base (por columnas), en el sistema de coordenadas original.
* $P_{B_C \rightarrow B}$ = $P_{B \rightarrow B_C}^{-1}$

(!) Todo cambio de base puede ser visto también como una transformación.

In [ ]:
# The Basis
v_1 = np.array([[1],[1]])/np.sqrt(2)
v_2 = np.array([[-1],[1]])/np.sqrt(2)

# Change of coordinate matrix
p_B_Bc =np.concatenate([v_1, v_2], axis=1)
p_inv =  np.linalg.inv(p) 

# Coordinate of x in R^2
x=np.array([[0], [np.sqrt(2)]])

# New coordinate relative to basis B
x_B = p_inv @ x

print("x_B=", np.round(x_B, 2))
print(p_B_Bc)

In [ ]:
print("x_B=", np.round(x_B, 2))
origin = [0], [0]
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,15))

plt.subplots_adjust(wspace=0.4)

# Plotting x in R2
ax1.quiver(*origin, x[0], x[1], color=['r'], width=0.01, angles='xy', scale_units='xy', scale=1)
ax1.quiver(*origin, 1, 0, color=['b'], width=0.01, angles='xy', scale_units='xy', scale=1)
ax1.quiver(*origin, 0, 1, color=['b'], width=0.01, angles='xy', scale_units='xy', scale=1)

ax1.set_xlim([-5,6])
ax1.set_ylim([-3,4])
ax1.set_aspect('equal')
ax1.grid(True)
ax1.set_title("Coordinate of $\mathbf{x}$ in $\mathbf{R^2}$")
ax1.axhline(y=0, color='k')
ax1.axvline(x=0, color='k')
ax1.text(x[0]+0.2, x[1]+0.2, "$\mathbf{x}$", fontsize=12)
ax1.text(1, -0.8, "$\mathbf{i}$", fontsize=12)
ax1.text(-0.6, 1, "$\mathbf{j}$", fontsize=12)
ax1.axvline(x=x[0], color='grey', linestyle='--')
ax1.axhline(y=x[1], color='grey', linestyle='--')

# Plotting x in B
# Plotting the grid
multipliers = np.linspace(-10,10,100)

for i in range(-6,7,4):
    for j in range(-6,7,1):
        grid_1 = (v_1 * multipliers) + np.array([[i],[j]])
        grid_2 = (v_2 * multipliers) + np.array([[i],[j]])
        ax2.plot(grid_1[0], grid_1[1], color='grey', linewidth=0.2)
        ax2.plot(grid_2[0], grid_2[1], color='grey', linewidth=0.2)
        
# Plotting the vector guide
vector_guide_1 = (v_1 * multipliers) + x
vector_guide_2 = (v_2 * multipliers) + x
ax2.plot(vector_guide_1[0], vector_guide_1[1], color='grey', linewidth=1.5, linestyle='--')
ax2.plot(vector_guide_2[0], vector_guide_2[1], color='grey', linewidth=1.5, linestyle='--')

# Plotting the axis
vector_guide_1 = (v_1 * multipliers) + np.array([[0],[0]])
vector_guide_2 = (v_2 * multipliers) + np.array([[0],[0]])
ax2.plot(vector_guide_1[0], vector_guide_1[1], color='black', linewidth=1.2)
ax2.plot(vector_guide_2[0], vector_guide_2[1], color='black', linewidth=1.2)

ax2.quiver(*origin, x[0], x[1], color=['r'], width=0.01, angles='xy', scale_units='xy', scale=1)
ax2.quiver(*origin, v_1[0], v_1[1], color=['b'], width=0.01, angles='xy', scale_units='xy', scale=1)
ax2.quiver(*origin, v_2[0], v_2[1], color=['b'], width=0.01, angles='xy', scale_units='xy', scale=1)

ax2.set_xlim([-5,6])
ax2.set_ylim([-3,4])
ax2.set_aspect('equal')

ax2.set_title("Coordinate of $\mathbf{x}$ in $\mathit{B}$")
ax2.axhline(y=0, color='k', linewidth=0.5)
ax2.axvline(x=0, color='k', linewidth=0.5)
ax2.text(x[0]+0.2, x[1]+0.2, "$\mathbf{x}$", fontsize=12)
ax2.text(1, 0.2, "$\mathbf{u_1}$", fontsize=12)
ax2.text(-0.7, 0.8, "$\mathbf{u_2}$", fontsize=12)

plt.show()

Esto es viendo P como la matriz de un cambio de base. Sin embargo, se podría haber visto como la transformación que lleva el vector $(0,\sqrt(2))$ en el vector $(1, 1)$, todo en la Base Canónica $B_C$. Cuando trabajamos con la base canónica, los cambios de base por lo general son un mierdón.

### Diagonalización de matrices simétricas

Ya hemos visto que podemos diagonalizarla de la forma:

$PDP^T$

donde las columnas de $P$ son los autovalores de $A$ (por columnas). De este modo $P^T = P^{-1}$ es un cambio de base o giro (dependiendo del punto de vista elegido), $D$ es la transformación sobre los ejes, y $P$ es el giro o cambio de base inverso.

De este modo, la transformación $A$ se descompone en un giro (o cambio de base) para que la base y autovectores coincidan, la transformación, y y luego lo dejamos tood como estaba.

### Autovalores y Autovectores de $AA^T$ y $A^TA$

Al ser matrices simétricas, son diagonalizables, con autovalores reales y autovectores ortogonales. `eig_vector` nos da valores normalizados siempre. Podemos tomarlos siempre así, y nos ayuda a apreciar su transformación.

In [ ]:
A = np.array([[1, 2], [3, 1]])
eig_values, eig_vectors = linalg.eig(A@A.T)
xs = eig_vectors
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
print(np.round(eig_values, 4))
print(np.round(eig_vectors, 4))

In [ ]:
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

Y vamos a ver que, en efecto, son ortogonales (esto es, $AA^T=I$).

In [ ]:
eig_vectors @ eig_vectors.T

In [ ]:
A = np.array([[1, 2], [3, 1]])
eig_values, eig_vectors = linalg.eig(A.T@A)
xs = eig_vectors.T
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T
print(np.round(eig_values, 4))
print(np.round(eig_vectors, 4))

In [ ]:
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

### Ejemplo matriz simétrica

Veamos un ejemplo de $A = PDP^T$.

Tenemos esta matriz simétrica

In [ ]:
A = [
    [3, 1],
    [1, 2]
]
A

Calculamos sus autovalores y autovectores, y formamos las matrices $P$ y $D$ correspondientes.

In [ ]:
eig_values, eig_vectors = linalg.eig(A)

In [ ]:
D = np.diag(eig_values)
D

In [ ]:
P = eig_vectors
P

Comprobamos que el resultado es el esperado.

In [ ]:
P @ D @ P.T

Vamos a comprobar visualmente, paso a pasao, como primero giramos, después transformamos, y hacemos el giro inverso.

Esta es la transformación completa:

In [ ]:
xs = eig_vectors.T
Axs = (A @ xs.T).T
S = get_unit_circle().T
f_S = (A @ S.T).T

In [ ]:
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

Ahora, paso por paso. Primero giramos:

In [ ]:
xs = eig_vectors.T
Axs = (P.T @ xs.T).T
S = get_unit_circle().T
f_S = (P.T @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

Aplicamos la transformación:

In [ ]:
xs = (P.T @ eig_vectors).T
Axs = (D @ P.T @ eig_vectors).T
S = get_unit_circle().T
f_S = (D @ P.T @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

Volvemos a girar:

In [ ]:
xs = (D @ P.T @ eig_vectors).T
Axs = (P @ D @ P.T @ eig_vectors).T
S = (D @ P.T @ get_unit_circle()).T
f_S = (P @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

## 4. Descomposición en valores singulares.

Vale, hasta ahora sabemos que si una matriz es cuadrada, igual hasta se puede diagonalizar. Y si encima son simétricas, eso es una maravilla, que ya hasta te quitas de calcular inversas ni movidas raras...

Ahora bien... ¿qué pasa cuando una matriz no es ni siquiera cuadrada? Ahí

En ese caso es donde entra en juego la descomposición en valores singulares. Si $A \in \mathbb{R}^{mxn}$, entonces podemos encontrar matrices tales que $$A = U\Sigma V^T$$ donde $\Sigma$ es diagonal.

### Troncho

Vamos a ver cómo llegamos a eso...

Al ser una matriz rectangular, nuestra aplicación pasa de un espacio de $m$ dimensiones a uno com $n$ dimensiones.

Si tuviésemos unos vectores buenos (que fuesen como autovectores) y pudiésemos formar una base con ellos, suponiéndolos unitarios, tendríamos

$$ V = {v_1, ..., v_m} $$

y cada uno de esos vectores, se estiraría o se encogería. Esto lo podemos expresar como una base unitaria, multiplicada por los autovalores correspondientes $U\Sigma$. Esto es:

$$Av_1 = u_1 \sigma_i$$
$$\vdots$$
$$Av_r = u_r \sigma_i$$
$$Av_{r+1} = 0$$
$$\vdots$$

donde $r$ es el rango de $A$. Entonces, tenemos

$$AV = U\Sigma$$

y claro, de aquí tenemos que, si la base fuese ortonormal, como esperamos...

$$A = U\Sigma V^T$$

Vale, hasta aquí estupendo, pero... ¿es esto posible?

Pues, sí que sabemos que $AA^T$ y $A^TA$ son simétricas, y por tanto son ortogonalmente diagonablizables. Además, atendiendo a la definición de A a la que aspiramos,

$$A^TA = V \Sigma^T U^T U \Sigma V^T =  V \Sigma^T \Sigma V^T = V \Sigma^2 V^T $$
$$AA^T = U \Sigma^T V^T V \Sigma U^T =  U \Sigma^T \Sigma U^T = U \Sigma^2 U^T $$

Y resulta que tanto $U$, como $V$ existen, y son ortogonales, porque $AA^T$ y $A^TA$ son ortogonalmente diagonalizables :)


Por cierto,

$$ A^TA, U \in \mathbb{R}^{nxn} $$
$$ AA^T, V \in \mathbb{R}^{mxm} $$

Un último comentario... nos podemos encontrar con que la expresión de SVD viene únicamente $r$ vectores en $U$ y $V$, y $\Sigma$ es cuadrada de $r \times r$, siendo $r$ el rango de $A$, o la expresión completa, donde $\Sigma$ tiene la misma dimensión que $A$, pero completando con ceros. Tanto $U$ como $V$ podrán tener más vectores, que pertenecerán al espacio nulo (o kernel) de cada espacio respectivamente, si existen.

### Ejemplo

Vamos a ver un ejemplo simple. En primer lugar vamos a usar numpy para calcular ambas formas:

In [ ]:
A = np.array([
    [3, 1],
    [1, 2],
    [1, 1]
])

Para obtener la forma completa, con los vectores del espacio nulo:

In [ ]:
np.linalg.svd(A, full_matrices=True)

Para obtener la expresión reducida:

In [ ]:
np.linalg.svd(A, full_matrices=False)

Vamos a repetir el ejemplo de la sección anterior.

In [ ]:
A = np.array([
    [3, 1],
    [1, 2]
])

In [ ]:
U, singular_values, VT = np.linalg.svd(A, full_matrices=True)

In [ ]:
sigma = np.diag(singular_values)

In [ ]:
U, sigma, VT

In [ ]:
eig_values, eig_vectors = np.linalg.eig(A)

Como en el ejemplo anterior, vamos a volver a hacerlo paso por paso:

In [ ]:
xs = eig_vectors.T
Axs = (VT @ xs.T).T
S = get_unit_circle().T
f_S = (VT @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

In [ ]:
xs = (VT @ eig_vectors).T
Axs = (sigma @ VT @ eig_vectors).T
S = get_unit_circle().T
f_S = (sigma @ VT @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()

In [ ]:
xs = (sigma @ VT @ eig_vectors).T
Axs = (U @ sigma @ VT @ eig_vectors).T
S = (sigma @ VT @ get_unit_circle()).T
f_S = (VT.T @ S.T).T
plot_2d_vectors(xs, Axs, (S, f_S))
plt.show()